In [2]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import os
from skimage import measure
from shapely.geometry import Polygon, MultiPolygon
pylab.rcParams['figure.figsize'] = (25, 25)

import lib.coco_handler as coco_handler
import json

In [14]:
dataDir='../data'
annFile= os.path.join(dataDir,"all_coco.json")
moths = COCO(annFile)

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [13]:
type(list(moths.anns.values()))

list

## COCO handling

In [4]:
#TODO create class to handle

def convert_bb_to_coco(minx, miny, maxx, maxy):
    """converts bb from shapely to coco format"""
    width = round(maxx - minx, 1)
    height = round(maxy - miny, 1)
    return round(minx), round(miny), width, height

def create_coco_anno_from_mask(mask, min_area, category_id = 3, annotation_id = 500, image_id=5):
    mask = np.flip(np.rot90(mask, 1), axis=0)
    contours = measure.find_contours(mask, 0.5, positive_orientation="low")
    contours = [c for c in contours if Polygon(c).area > min_area]
    is_crowd = 0
    a = []
    for contour in contours:
        # Make a polygon and simplify it
        poly = Polygon(contour)
        if poly.area < min_area:
            continue
        poly = poly.simplify(1.0, preserve_topology=False)
        segmentation = [np.array(poly.exterior.coords).ravel().tolist()]
        d = {
            "id": annotation_id,
            "image_id": image_id,
            "category_id": category_id,
            "iscrowd": is_crowd,
            "area": poly.area,
            "bbox": [*convert_bb_to_coco(*poly.bounds)],
            "segmentation": segmentation,
            }
        a.append(d)
        annotation_id += 1
    return a

In [5]:
min_bug_area = 500

mask = io.imread(os.path.join(dataDir,"masks", "bug_test.tif"))
img = io.imread(os.path.join(dataDir,"imgs", "bug_test.tif"))
contours = measure.find_contours(mask, 0.5, positive_orientation='low')
contours = [c for c in contours if Polygon(c).area > min_bug_area]


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\sebas\\Documents\\PG ISS SS21\\data\\coco_anno\\masks\\bug_test.tif'

In [ ]:
f, axs = plt.subplots(1,3, figsize=(10,8))

axs[0].imshow(img, cmap='gray')


axs[1].imshow(mask, cmap='gray')
axs[2].imshow(img, cmap='gray')

for contour in contours:
    axs[2].plot(contour[:, 1], contour[:, 0], linewidth=2)

axs[2].axis('image')
axs[2].set_xticks([])
axs[2].set_yticks([])

plt.show()

In [ ]:
anno1 = create_coco_anno_from_mask(mask, min_bug_area)

In [ ]:
min_bug_area = 500

mask = io.imread(os.path.join(dataDir,"masks", "kasten_8.tif"))
img = io.imread(os.path.join(dataDir,"imgs", "kasten_8.tif"))
contours = measure.find_contours(mask, 1, positive_orientation='low')
contours = [c for c in contours if Polygon(c).area > min_bug_area]

f, axs = plt.subplots(1,3, figsize=(10,8))

axs[0].imshow(img, cmap='gray')
axs[0].set_xticks([])
axs[0].set_yticks([])

axs[1].imshow(mask, cmap='gray')
axs[1].set_xticks([])
axs[1].set_yticks([])

axs[2].imshow(img, cmap='gray')

for contour in contours:
    axs[2].plot(contour[:, 1], contour[:, 0], linewidth=2)

axs[2].axis('image')
axs[2].set_xticks([])
axs[2].set_yticks([])

plt.show()

anno2 = create_coco_anno_from_mask(mask, 500, category_id = 4, annotation_id = 600, image_id=6)


In [ ]:
coco_anno = {
    **coco_handler.coco_dataset.coco_info,
    **coco_handler.coco_dataset.coco_licenses,
    **coco_handler.coco_dataset.coco_images,
    "annotations":anno1+anno2
    }

In [ ]:
with open('../../data/coco_anno/bugs.json', 'w') as f:
    f.write(json.dumps(coco_anno))


In [ ]:
bugs = COCO("../../data/coco_anno/bugs.json")

In [ ]:
annIds = bugs.getAnnIds(imgIds=[6], catIds=range(5), iscrowd=None)
anns = bugs.loadAnns(annIds)
print(len (anns))
plt.axis('off')
plt.imshow(img, cmap="gray")
bugs.showAnns(anns, True)
plt.show()

# other code


In [ ]:
coco = COCO("../data/anno/all_anno.json")
data_dir = ("../data/imgs/")
image= coco.imgs[3]

In [ ]:
out_dir = "../output/segmented_bugs/"
for _, cat in coco.cats.items():
    os.makedirs(os.path.join(out_dir, cat["name"]), exist_ok=True)


In [ ]:
img = Image.open(os.path.join(data_dir, image["file_name"]))
img = img.convert("RGBA")
img = np.asarray(img.convert('RGBA'))

In [ ]:
anns_ids = coco.getAnnIds(imgIds=image["id"], iscrowd=None)
anns = coco.loadAnns(anns_ids)
an = anns[0]
xmin, ymin, w, h = tuple([int(i) for i in an["bbox"]])
mask = coco.annToMask(an)[ymin-1:ymin+h+2,xmin-1:xmin+w+2]
obj = img[ymin-1:ymin+h+2,xmin-1:xmin+w+2].copy()
obj[:, :, 3] = np.where(mask, obj[:, :, 3], np.nan)

plt.imshow(obj)
cat = coco.loadCats(ids=[an["category_id"]])[0]["name"]
Image.fromarray(obj).save(os.path.join("../data/segmented_bugs", cat, "test.png"))



In [ ]:
anns_ids = coco.getAnnIds(imgIds=image["id"], iscrowd=None)
anns = coco.loadAnns(anns_ids)

for i, an in enumerate(anns):
    img = Image.open(os.path.join(data_dir, image["file_name"]))
    img = img.convert("RGBA")
    img = np.asarray(img.convert('RGBA'))

    xmin, ymin, w, h = tuple([int(i) for i in an["bbox"]])
    mask = coco.annToMask(an)[ymin-1:ymin+h+2,xmin-1:xmin+w+2]
    obj = img[ymin-1:ymin+h+2,xmin-1:xmin+w+2].copy()
    obj[:, :, 3] = np.where(mask, obj[:, :, 3], np.nan)

    plt.imshow(obj)
    cat = coco.loadCats(ids=[an["category_id"]])[0]["name"]
    Image.fromarray(obj).save(os.path.join("../output/segmented_bugs", cat, f"{i}.png"))
    plt.show()

In [ ]:
cat_ids = coco.getCatIds()
for image in coco.imgs:
    img = Image.open(os.path.join(data_dir, image["file_name"]))
    img = img.convert("RGBA")

    anns_ids = coco.getAnnIds(imgIds=img["id"], iscrowd=None)
    anns = coco.loadAnns(anns_ids)
    for an in anns:
        mask = coco.annToMask(an)
        obj = np.where(mask, img, np.nan)
        io.imsave(os.path.join(out_dir, an["category_id"]))


In [ ]:
# cat_ids = coco.getCatIds()
# anns_ids = coco.getAnnIds(imgIds=img["id"], catIds=cat_ids, iscrowd=None)
# anns = coco.loadAnns(anns_ids)
# anns_img = np.zeros((img['height'],img['width']))
# for ann in anns:
#     anns_img = np.maximum(anns_img,coco.annToMask(ann)*ann['category_id'])